In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from collections import OrderedDict
from scipy.ndimage import shift as shift_fn


from global_shift import to_tensor, data_load, shift_adjustment,shift_image_subpixel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.Size([1, 3, 3456, 4608])
(-17.10400640713806, 1.0672462433888086)
(-0.10567152960038584, 0.07038063961977059)


In [2]:

def CHbayer_to_bayer(bayer_array):
    H, W = bayer_array.shape[1:]
    bayer = np.zeros((H * 2, W * 2), dtype=bayer_array.dtype)
    bayer[0::2, 0::2] = bayer_array[0]
    bayer[0::2, 1::2] = bayer_array[1]
    bayer[1::2, 0::2] = bayer_array[2]
    bayer[1::2, 1::2] = bayer_array[3]

    return bayer

In [3]:
def phase_correlation_shift(img1, img2):
    shift, _ = cv2.phaseCorrelate(img1, img2)
    return shift

In [12]:
def global_shift(mid, gt):
    shifted_gt_list = []

    for b in range(gt.shape[0]):
        mid_numpy = mid[b][:4, :, :].squeeze().cpu().numpy()
        gt_numpy = gt[b].squeeze().cpu().numpy()

        mid_full = CHbayer_to_bayer(mid_numpy)
        gt_full = CHbayer_to_bayer(gt_numpy)

        print(mid_full.shape)
        initial_shift = phase_correlation_shift(mid_full, gt_full)
        shifted_gt = shift_image_subpixel(gt_full, initial_shift) #(H,W,C)
        after_shift = phase_correlation_shift(mid_full, shifted_gt)
        #print(to_tensor(shifted_gt).unsqueeze(0).shape)
        shifted_gt_tensor = to_tensor(shifted_gt).unsqueeze(0)  # Should be (C,H,W)
        shifted_gt_list.append(shifted_gt_tensor)

    shifted_gt_batch = torch.stack(shifted_gt_list, dim=0)  # (B, C, H, W)

    return shifted_gt_batch, initial_shift, after_shift


In [14]:
npypath = '/data/asim/ISP/HDR_transformer/data/RAW/2022-0708-1630-2434.npz'
mid, gt = data_load(npypath)
print(mid.shape)
print(gt.shape)
shifted_gt_rgb, intial_shift, after_shift = global_shift(mid, gt)
shifted_gt_rgb.shape, intial_shift, after_shift

torch.Size([1, 8, 1728, 2304])
torch.Size([1, 4, 1728, 2304])
(3456, 4608)


(torch.Size([1, 1, 3456, 4608]),
 (-17.530512806594288, 0.9916706411631822),
 (0.4701037762620217, -0.005247024676691581))